# Embeddings検証用のデータセット作成
- ライブドアニュース、9カテゴリ、各300件、計2700件、戦闘から512文字を抽出
- HuggingFaceの`shunk031/livedoor-news-corpus`を使用する
- `livedoor_300.csv.zip`として保存

In [ ]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset(
    "shunk031/livedoor-news-corpus", ## HuggingFaceのDatasets名
    train_ratio=0.998, ## 1だとエラーになるので
    val_ratio=0.001,
    test_ratio=0.001,
    random_state=42, 
    shuffle=False,
)

name = 'train' ## trainのみ収集
df = pd.DataFrame({'category':dataset[name]["category"], 'content':dataset[name]["content"], 'title':dataset[name]["title"]})

## 各カテゴリ別のデータ数表示
for i in df['category'].unique():
    print(i, len(df[df['category']==i]))

# 特定のカラムの文字数を512文字でカットする関数を定義する
def cut_text(text):
    if len(text) > 512:
        return text[:512]
    else:
        return text

# 特定のカラムの文字列を512文字でカットする
target_column = 'content512' ## 保存カラム名
df[target_column] = df['content'].apply(cut_text)
df = df.reset_index()

## 必要なカラムだけ残す
df = df[['index','category','content512']]

## 各categoryごとに300件ずつ抽出して追加する
df_save = pd.DataFrame()
for i in range(9): ## categoryは0-9
    if i == 0:
        df_save = df[df['category']==i][0:300]
    else:
        df_save = pd.concat([df_save, df[df['category']==i][0:300]], axis=0)

## 保存
df_save.to_csv('livedoor_300.csv.zip', compression="zip")

print('saved')
